# Прогноз изменения цен на недвижимость

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

Функция для вычисления RMSLE

In [ ]:
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean((np.log1p(y_true) - np.log1p(y_pred))**2))

Функция обработки пропущенных значений

In [ ]:
def zapolnenie_propuskov(df):
    df = df.copy()

    # Заполняем пропуски
    df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].median())

    # Для категориальных - заполняем модой
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        df[col] = df[col].fillna(df[col].mode()[0] if len(df[col].mode()) > 0 else 'None')

    # Для числовых - заполняем медианой
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())

    return df

Функция добавления новых признаков

In [ ]:
def new_features(df):
    df = df.copy()

    # ОБЩАЯ ПЛОЩАДЬ
    df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']

    # ПЛОЩАДЬ НА КОМНАТУ
    df['SFPerRoom'] = df['GrLivArea'] / (df['TotRmsAbvGrd'] + 1)

    # ОБЩЕЕ КОЛИЧЕСТВО ВАННЫХ
    df['TotalBathrooms'] = (df['FullBath'] + 0.5 * df['HalfBath'] +
                           df['BsmtFullBath'] + 0.5 * df['BsmtHalfBath'])

    # ОБЩАЯ ПЛОЩАДЬ КРЫЛЬЦА
    df['TotalPorchSF'] = (df['OpenPorchSF'] + df['EnclosedPorch'] + df['3SsnPorch'] + df['ScreenPorch'])

    # БИНАРНЫЕ ПРИЗНАКИ
    df['HasPool'] = (df['PoolArea'] > 0).astype(int)
    df['HasGarage'] = (df['GarageArea'] > 0).astype(int)
    df['HasBasement'] = (df['TotalBsmtSF'] > 0).astype(int)
    df['HasFireplace'] = (df['Fireplaces'] > 0).astype(int)
    df['Has2ndFloor'] = (df['2ndFlrSF'] > 0).astype(int)

    # ВОЗРАСТ ДОМА
    df['HouseAge'] = df['YrSold'] - df['YearBuilt']
    df['RemodAge'] = df['YrSold'] - df['YearRemodAdd']

    # ОБЩЕЕ КАЧЕСТВО
    df['OverallGrade'] = df['OverallQual'] * df['OverallCond']

    # ПЛОЩАДЬ КУХНИ ОТНОСИТЕЛЬНО ОБЩЕЙ
    df['KitchenRatio'] = df['KitchenAbvGr'] / (df['TotRmsAbvGrd'] + 1)

    return df

Функция обработки аномальных значений

In [ ]:
def remove_out(df):
    df = df.copy()

    # Удаляем дома с аномально большой жилой площадью но низкой ценой
    df = df[~((df['GrLivArea'] > 3500) & (df['SalePrice'] < 325000))]

    # Удаляем дома с аномально низкой общей площадью но высокой ценой
    df = df[~((df['TotalSF'] < 1000) & (df['SalePrice'] > 325000))]

    # Удаляем выбросы по качеству
    df = df[~((df['OverallQual'] == 10) & (df['SalePrice'] < 200000))]

    return df

Загрузка данных

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Обработка данных

In [ ]:
# Сохраняем Id для сабмита
test_ids = test['Id']

# Применяем заполнение пропусков ко всем данным
train_ZP = zapolnenie_propuskov(train)
test_ZP = zapolnenie_propuskov(test)

# Создание новых признаков
train_new = new_features(train_ZP)
test_new = new_features(test_ZP)

# Удаление выбросов в тренировочных данных
train_RO = remove_out(train_new)

print(f"Количество удалённых выбросов: {len(train_new)-len(train_RO)}")

# Разделяем тренировочные данные на train/validation ДО обработки
train_data, val_data = train_test_split(train_RO, test_size=0.2, random_state=17, shuffle=True)

print(f"Размер train_data: {train_data.shape}")
print(f"Размер val_data: {val_data.shape}")

# Объединяем ВСЕ данные для согласованной обработки (train_data + val_data + test)
all_data = pd.concat([train_data.drop('SalePrice', axis=1),
                      val_data.drop('SalePrice', axis=1),
                      test_new], axis=0, ignore_index=True)

# Логарифмируем целевую переменную для train_data и val_data
y_train = np.log1p(train_data['SalePrice'])
y_val = np.log1p(val_data['SalePrice'])

# Сохраняем оригинальные цены для validation (для вычисления RMSLE)
y_val_original = val_data['SalePrice']

# Базовое кодирование категориальных переменных
all_data_encoded = pd.get_dummies(all_data)

# Разделяем обратно на train, validation и test
X_train = all_data_encoded.iloc[:len(train_data)]
X_val = all_data_encoded.iloc[len(train_data):len(train_data) + len(val_data)]
X_test = all_data_encoded.iloc[len(train_data) + len(val_data):]

print(f"\nПосле обработки:")
print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_val: {y_val.shape}")

Количество удалённых выбросов: 2
Размер train_data: (1166, 94)
Размер val_data: (292, 94)

После обработки:
X_train: (1166, 300)
X_val: (292, 300)
X_test: (1459, 300)
y_train: (1166,)
y_val: (292,)


Создание и обучение модели

In [ ]:
# Базовая модель
#model = GradientBoostingRegressor(n_estimators=100,random_state=42)
model = GradientBoostingRegressor(n_estimators=500 , learning_rate=0.1, max_depth=4, min_samples_split=20, min_samples_leaf=10, subsample=0.8, random_state=17)

# Обучаем на тренировочных данных
model.fit(X_train, y_train)

# Предсказываем логарифмы цен на validation данных и преобразуем обратно к исходным ценам
y_val_pred_log = model.predict(X_val)
y_val_pred = np.expm1(y_val_pred_log)

# Вычисляем RMSLE на VALIDATION
val_rmsle = rmsle(y_val_original, y_val_pred)
print(f"\n=== РЕЗУЛЬТАТЫ ВАЛИДАЦИИ ===")
print(f"RMSLE на validation данных: {val_rmsle:.5f}")

# Теперь обучаем модель на ВСЕХ тренировочных данных
print(f"\n=== ФИНАЛЬНОЕ ОБУЧЕНИЕ ===")
X_all_train = pd.concat([X_train, X_val])
y_all_train = np.concatenate([y_train, y_val])

#final_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
final_model = GradientBoostingRegressor(n_estimators=500 , learning_rate=0.1, max_depth=4, min_samples_split=20, min_samples_leaf=10, subsample=0.8, random_state=17)
final_model.fit(X_all_train, y_all_train)
print("Финальная модель обучена на всех данных!")


=== РЕЗУЛЬТАТЫ ВАЛИДАЦИИ ===
RMSLE на validation данных: 0.12599

=== ФИНАЛЬНОЕ ОБУЧЕНИЕ ===
Финальная модель обучена на всех данных!


Предсказание на тестовых данных

In [ ]:
# Предсказание на тестовых данных
y_test_pred_log = final_model.predict(X_test)
predictions = np.expm1(y_test_pred_log)

# Создание файла для отправки
submission = pd.DataFrame({
    'Id': test_ids,
    'SalePrice': predictions
})
submission.to_csv('submission7.csv', index=False)

print(f"\n=== СТАТИСТИКА ПРЕДСКАЗАНИЙ ===")
print(f"Минимальная цена: ${predictions.min():,.0f}")
print(f"Максимальная цена: ${predictions.max():,.0f}")
print(f"Средняя цена: ${predictions.mean():,.0f}")
print(f"Медианная цена: ${np.median(predictions):,.0f}")


=== СТАТИСТИКА ПРЕДСКАЗАНИЙ ===
Минимальная цена: $43,164
Максимальная цена: $639,416
Средняя цена: $179,354
Медианная цена: $157,644
